# Taller de modelos de clasificación utilizando KNN

El objetivo es que puedan asimilar el proceso de aprendizaje de modelos de aprendizaje supervisado (en este caso de clasificación). En este taller encontrarán código que deberán reproducir en sus propios ambientes de desarrollo. Incluye preguntas precisas; para responderlas tendrán que ejecutar el código, pues no todas las porciones de código del taller muestran los resultados.

# 1. Análisis exploratorio (muy básico…)

# Librerías a importar

In [ ]:
import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn import tree, datasets, neighbors, metrics
from sklearn.metrics import confusion_matrix

#from sklearn import neighbors, datasets, metrics
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import cross_val_score, cross_validate #método para evaluar varios particionamientos de C-V
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, LeaveOneOut #Iteradores de C-V
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import math

import warnings
warnings.filterwarnings('ignore')

# Cargar y explorar el dataset de defaults de churn de clientes

In [ ]:
data = pd.read_csv('02-churn.csv', sep=';', na_values=".")
print(data.shape)
data.head(5)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.LEAVE.describe()

In [ ]:
data.LEAVE.describe()['freq'] / data.LEAVE.describe()['count']

In [ ]:
data.LEAVE.value_counts()

In [ ]:
data.LEAVE.value_counts().plot.bar()

PREGUNTA: ¿Qué ven de particular en los datos?

# Repaso de acceso a características y datos de un dataframe

In [ ]:
data.columns

In [ ]:
data.iloc[0:4,]

In [ ]:
data.iloc[0:4,1:4]

In [ ]:
data.LEAVE.iloc[1:5]

In [ ]:
data.iloc[0:10,[0,4]]

In [ ]:
data.loc[0:10,['COLLEGE','HOUSE']]

In [ ]:
data.values

# Visualización de las distribuciones de las variables independientes.

# Univariadamente

In [ ]:
plt.figure(figsize=(15,10))
sns.kdeplot(data[data['LEAVE']=='LEAVE']['HOUSE'], shade=True, color='r')
sns.kdeplot(data[data['LEAVE']=='STAY']['HOUSE'], shade=True, color='g')
plt.legend(['LEAVE', 'STAY'])

In [ ]:
plt.figure(figsize=(15,10))
sns.kdeplot(data[data['LEAVE']=='LEAVE']['OVERAGE'], shade=True, color='r')
sns.kdeplot(data[data['LEAVE']=='STAY']['OVERAGE'], shade=True, color='g')
plt.legend(['LEAVE', 'STAY'])

In [ ]:
plt.figure(figsize=(15,10))
sns.kdeplot(data[data['LEAVE']=='LEAVE']['INCOME'], shade=True, color='r')
sns.kdeplot(data[data['LEAVE']=='STAY']['INCOME'], shade=True, color='g')
plt.legend(['LEAVE', 'STAY'])

In [ ]:
plt.figure(figsize=(15,10))
sns.kdeplot(data[data['LEAVE']=='LEAVE']['LEFTOVER'], shade=True, color='r')
sns.kdeplot(data[data['LEAVE']=='STAY']['LEFTOVER'], shade=True, color='g')
plt.legend(['LEAVE', 'STAY'])

# Bivariadamente

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="HOUSE", y="OVERAGE", hue="LEAVE", data=data)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="HOUSE", y="INCOME", hue="LEAVE", data=data)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="INCOME", y="OVERAGE", hue="LEAVE", data=data)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="LEFTOVER", y="OVERAGE", hue="LEAVE", data=data)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="HOUSE", y="REPORTED_SATISFACTION", hue="LEAVE", data=data)

# Pairplots

In [ ]:
plt.figure(figsize=(15,15))
sns.pairplot(data,hue='LEAVE')

# Boxplots

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x="INCOME", y="LEAVE", data=data)

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x="LEFTOVER", y="LEAVE", data=data)

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x="HOUSE", y="LEAVE", data=data)

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x="OVERAGE", y="LEAVE", data=data)

# 2. Proceso completo (Particionamiento + normalización + modelos KNN)

## Protocolos de evaluación

Vamos ahora a evaluar los modelos que calculamos con diferentes protocolos de evaluación para tener una idea más clara de la calidad de los mismos, e identificar posibles casos de modelos que sufren de overfitting (sobreaprendizaje).

In [ ]:
data.shape[0] #Se tienen 20000 filas

Obtenemos las variables independientes numéricas y la variable dependiente

In [ ]:
numericVars = data.iloc[:,1:8]
depVar = data['LEAVE']

## Holdout (split)

Vamos a separar el dataset en 2 partes: 75% de los datos se van a utilizar para aprender, 25% para evaluar el modelo de clasificación. Utilizamos el método train_test_split de scikit-learn, que se encarga de hacer el particionamiento aleatorio:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(numericVars, depVar, random_state=1234, test_size = 0.25)

Los parámetros de este método son:
- train_size o test_size: define la proporción del dataset que se irán al training set o al test set.
- random_state: define la **semilla** a utilizar para incializar el generador de números pseudo-aleatorios. Se requiere que los resultados obtenidos con la partición sean eventualmente reproducibles. La semilla aleatoria debe inicalizarse en el mismo valor para obtener los mismos resultados.
- stratify: indica un array con los valores de una variable que se quiere tener en cuenta en el particionamiento, de tal manera que las proporciones originales se conserven después de la partición.

In [ ]:
X_train.shape

Vamos a reescalar las variables predictivas para que tengan la misma importancia, siguiendo un proceso de estandarización.

In [ ]:
x = X_train.values
x_std = StandardScaler().fit_transform(x)
X_train_std = pd.DataFrame(x_std)

In [ ]:
x = X_test.values
x_std = StandardScaler().fit_transform(x)
X_test_std = pd.DataFrame(x_std)

Entrenamos varios modelos knn para encontrar el k más apropiado.

In [ ]:
acc_train_vec=[]
acc_test_vec=[]
k_vec= np.arange(1,31,2)
for k in k_vec:
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_std, y_train)
    y_pred = knn.predict(X_train_std)
    acc_train_vec.append(metrics.accuracy_score(y_train, y_pred))
    y_pred = knn.predict(X_test_std)
    acc_test_vec.append(metrics.accuracy_score(y_test, y_pred))
print("Exactitud promedio (entrenamiento): %0.2f" % np.mean(acc_train_vec))    
print("Exactitud promedio (prueba): %0.2f" % np.mean(acc_train_vec)) 

# Find the maximum accuracy and its index
max_acc = max(acc_test_vec)
max_acc_index = acc_test_vec.index(max_acc)

# Get the corresponding k value
k_best = k_vec[max_acc_index]

print(f"Maximum test accuracy: {max_acc:.4f}")
print(f"Corresponding k value: {k_best}")

In [ ]:
plt.figure(figsize=(10,5))
ax = plt.gca() # get current axis
plt.plot(k_vec, acc_train_vec)
plt.plot(k_vec, acc_test_vec)
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.axis('tight')
plt.xlabel('k')
plt.ylabel('accuracy')
plt.title('Evolución de le exactitud vs complejidad del modelo k-nn (valor de k más pequeño)')
plt.legend(['train', 'test'])

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=27)
knn.fit(X_train_std, y_train)
y_preds = knn.predict(X_train_std)
print("Clases reales   : ", y_train)
print("Clases predichas: ", y_preds)

Obtenemos la matriz de confusión sobre el mismo conjunto de entrenamiento.

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_train, y_preds)

# Plot using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de confusion para k=27')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

Encontramos los valores de exactitud, kappa, precisión, sensibilidad, especificidad y F1-score sobre los datos de entrenamiento

In [ ]:
print("Exactitud: ", metrics.accuracy_score(y_train, y_preds))
print("Kappa    : ", metrics.cohen_kappa_score(y_train, y_preds))
print("Precisión     : ", metrics.precision_score(y_train, y_preds, labels=['LEAVE'], average='macro'))
print("Recall        : ", metrics.recall_score(y_train, y_preds, labels=['LEAVE'], average='macro'))
VN = cm[1,1]
FP = cm[1,0]
specificity = VN/(VN+FP)
print("Especificidad : ", specificity)
print("F1-score      : ", metrics.f1_score(y_train, y_preds, labels=['LEAVE'], average='macro'))

In [ ]:
y_preds = knn.predict(X_test_std)
print("Clases reales   : ", y_test)
print("Clases predichas: ", y_preds)

Obtenemos la matriz de confusión sobre el conjunto de prueba.

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_preds)

# Plot using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de confusion para k=27')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

Encontramos los valores de exactitud, kappa, precisión, sensibilidad, especificidad y F1-score sobre los datos de prueba

In [ ]:
print("Exactitud: ", metrics.accuracy_score(y_test, y_preds))
print("Kappa    : ", metrics.cohen_kappa_score(y_test, y_preds))
print("Precisión     : ", metrics.precision_score(y_test, y_preds, labels=['LEAVE'], average='macro'))
print("Recall        : ", metrics.recall_score(y_test, y_preds, labels=['LEAVE'], average='macro'))
VN = cm[1,1]
FP = cm[1,0]
specificity = VN/(VN+FP)
print("Especificidad : ", specificity)
print("F1-score      : ", metrics.f1_score(y_test, y_preds, labels=['LEAVE'], average='macro'))

PREGUNTA: Intentemos el mismo proceso con otra semilla (otro particionamiento), por ejemplo con 3457. ¿Qué opinan de los resultados de los dos modelos? ¿Cómo explican las diferencias?

## K-fold cross-validation

Este protocolo de evaluación consiste en dividir el dataset en K pedazos de igual tamaño, y analizar el rendimiento de un modelo aprendido que va rotando sobre k-1 subconjuntos y evaluado en el subconjunto faltante (El K del K-fold no tiene niguna relación con el K del K-NN). 
En el caso de clasificación, particionamiento se hace aleatoriamente y de manera estratificada con respecto a la variable objetivo.
Las métricas finales son las agregaciones de las evaluaciones de los K modelos.

#### cross_val_score

*scikit-learn* cuenta con una función que permite repetir el proceso de particionamiento y evaluación del K-fold CV. Se trata de **cross_val_score**, que recibe los siguientes parámetros:
- la instancia del modelo que se quiere evaluar, 
- los datos de las variables independiente, 
- los datos reales de la variable dependiente, 
- cv: el número de veces que se va a repetir el proceso de cross-validation
- scoring: la métrica que se desea evaluar

In [ ]:
x = numericVars.values
x_std = StandardScaler().fit_transform(x)
X_std = pd.DataFrame(x_std)
y = depVar

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=27)
exactitudes = cross_val_score(knn, X_std, y, cv=10, scoring='accuracy')
exactitudes

Vemos que los scores de las 10 iteraciones del CV dan resultados entre 66.6% y 69.9%. Podemos obtener un intervalo de confianza del 95% para estimar el valor de la exactitud generalizada.

In [ ]:
print("Exactitudes: %0.2f (+/- %0.2f)" % (exactitudes.mean(), exactitudes.std() * 2))

#### cross_validate

El problema es que con este método solo se puede evaluar una sola métrica a la vez, y que debe ser una métrica global, o tratar una clasificación binaria.

El método **cross_validate** permite evaluar mas de una métrica a la vez, pero en el caso de categorías que no sean binarias, las métricas de precision, recall y f1 son agregadas. La salida de este método es un directorio con las métricas resultantes, que además incluye el tiempo de aprendizaje y de evaluación de cada iteración.

In [ ]:
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
knn = neighbors.KNeighborsClassifier(n_neighbors=27)
scores = cross_validate(knn, X_std, y, scoring=scoring, cv=10, return_train_score=False)

for key in scores:
    score = scores[key]
    print("%s: %0.2f (+/- %0.2f)" % (key, score.mean(), score.std() * 2))

#### Iteradores de cross-validation: KFold, StratifiedKFold, LeaveOneOut

Podemos utilizar también clases específicas para los particionamientos de los datos que permiten mucha más flexibilidad. Las clases **KFold**, **RepeatedKFold**, y **LeaveOneOut** se limitan a crear iteradores que retornan los subconjuntos de training y test.

Es importante anotar que estos iteradores parten del supuesto de independencia de los registros, por lo que es necesario barajarlos previamente.

KFold solo particiona los datos en subconjuntos de items.

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=27)
kf = KFold(n_splits=10, shuffle=True)
acc_test_vec=[]

In [ ]:
for indices_train, indices_test in kf.split(X_std):
    #print("%s %s" % (indices_train, indices_test))
    knn.fit(X_std.iloc[indices_train], y.iloc[indices_train])
    y_pred = knn.predict(X_std.iloc[indices_test])
    acc_test_vec.append(metrics.accuracy_score(y.iloc[indices_test], y_pred))  
acc_test_vec

Un caso particular es cuando el K del KFold es igual al tamaño de la muestra. En tal caso, se obtiene un protocolo de LeaveOneOut. En este caso los resultados para cada test set (de tamaño 1) solo pueden ser del 100% o del 0%.

### No correr este

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=27)
loocv = LeaveOneOut()
acc_test_vec=[]
for indices_train, indices_test in loocv.split(X_std):
    #print("%s %s" % (indices_train, indices_test))
    knn.fit(X_std.iloc[indices_train], y.iloc[indices_train])
    y_pred = knn.predict(X_std.iloc[indices_test])
    acc_test_vec.append(metrics.accuracy_score(y[indices_test], y_pred))  
np.mean(acc_test_vec)

Una mejora se logra con el StratifiedKFold, pues se tiene en cuenta las proporciones de la variable objetivo en la partición, controlando un poco un posible sesgo en la aleatoriedad.

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=27)
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
acc_test_vec=[]
for indices_train, indices_test in kf.split(X_std, y):
    knn.fit(X_std.iloc[indices_train], y.iloc[indices_train])
    y_pred = knn.predict(X_std.iloc[indices_test])
    acc_test_vec.append(metrics.accuracy_score(y.iloc[indices_test], y_pred))  
acc_test_vec

In [ ]:
X_train_std.shape

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

knn = neighbors.KNeighborsClassifier(n_neighbors=27)
sfs = SequentialFeatureSelector(knn, k_features=5,        # Instead of n_features_to_select
                              forward=True,        # True for forward selection
                              floating=False,      
                              scoring='accuracy',
                              cv=5)
sfs.fit(X_train_std,y_train)
X_fs = sfs.transform(X_train)

In [ ]:
X_fs.shape

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=27)
knn.fit(X_fs,y_train)
preds = knn.predict(X_fs)
metrics.accuracy_score(y_train,preds)